In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S6'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 19998
Total Tokens Found = 292928
Total Tokens Output = 292542
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 19998
Total Tokens= 292542
Unique Tokens= 35953
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 19998
Sorted Token Size = 35953
Clean Token Size = 14728
*Stop Replacements here
References Processed =  19998
Total Replacement Pairs = 4726
Tokens Read = 292542
Tokens Changed =  5576
References Changed = 4876


In [2]:
status="done"

In [3]:
print(status)

done


In [14]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 15739
NewMatrix Iterateblocks Total Time = 21.94733450561762


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 13751
NewMatrix Iterateblocks Total Time = 19.306732788681984


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 10768
NewMatrix Iterateblocks Total Time = 13.027660648338497


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 6726
NewMatrix Iterateblocks Total Time = 10.656971072778106


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix 

0.65086943

Create functions for variable transformation.

In [8]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [9]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [13]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [14]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [15]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [16]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


0.6228630754264552

In [17]:
optimizer=None

In [20]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(500)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(500)),
           'mu_logit': (logit(0.5), logit(0.8)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(500)),
           'epsilonIterate_logit': (logit(0), logit(0.25))}

In [21]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [22]:
optimizer.probe(
    params={"beta_sp": inv_softplus(35), 
            "sigma_sp": inv_softplus(403),
            "mu_logit":logit(0.5),
            'epsilonIterate_logit': logit(0),
            "muIncr_logit":logit(0.1),
            "epsilon_sp":inv_softplus(15.1)},
    lazy=True,
)

In [23]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
)

|   iter    |  target   |  beta_sp  | epsilo... | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.10|Beta:35.00|Sigma:403.00|Epsilon:15.10|EpIter:0.00
|  1        |  2.217    |  35.0     | -124.3    |  15.1     | -2.197    |  0.0      |  403.0    |
Mu:0.55|Mu+:0.03|Beta:208.00|Sigma:46.00|Epsilon:1.04|EpIter:0.00
|  2        | -6.875    |  208.8    | -35.57    |  0.5985   | -3.625    |  0.2034   |  46.66    |
Mu:0.64|Mu+:0.05|Beta:93.00|Sigma:342.00|Epsilon:198.71|EpIter:0.00
|  3        | -6.56     |  93.57    | -81.75    |  198.7    | -2.866    |  0.5811   |  342.8    |
Mu:0.64|Mu+:0.08|Beta:102.00|Sigma:279.00|Epsilon:14.22|EpIter:0.00
|  4        |  2.129    |  102.7    | -16.12    |  14.22    | -2.444    |  0.5785   |  279.6    |
Mu:0.61|Mu+:0.18|Beta:70.00|Sigma:346.00|Epsilon:400.48|EpIter:0.00
|  5        | -6.874    |  70.66    | -99.93    |  400.5    | -1.488    |  

|  36       |  1.285    |  335.7    | -49.27    |  24.47    | -1.476    |  0.9198   |  382.5    |
Mu:0.69|Mu+:0.04|Beta:30.00|Sigma:400.00|Epsilon:13.21|EpIter:0.00
|  37       |  2.464    |  30.35    | -112.7    |  13.21    | -3.135    |  0.8129   |  400.7    |
Mu:0.59|Mu+:0.06|Beta:145.00|Sigma:160.00|Epsilon:10.10|EpIter:0.00
|  38       |  1.287    |  145.1    | -102.6    |  10.1     | -2.802    |  0.3643   |  160.1    |
Mu:0.61|Mu+:0.02|Beta:323.00|Sigma:395.00|Epsilon:25.21|EpIter:0.00
|  39       |  1.186    |  323.9    | -30.38    |  25.21    | -3.945    |  0.4679   |  395.0    |
Mu:0.58|Mu+:0.02|Beta:44.00|Sigma:405.00|Epsilon:27.45|EpIter:0.00
|  40       |  0.8149   |  44.0     | -110.1    |  27.45    | -3.679    |  0.3129   |  405.0    |
Mu:0.78|Mu+:0.03|Beta:331.00|Sigma:378.00|Epsilon:17.30|EpIter:0.00
|  41       |  1.787    |  332.0    | -37.31    |  17.3     | -3.65     |  1.294    |  378.4    |
Mu:0.55|Mu+:0.20|Beta:106.00|Sigma:288.00|Epsilon:15.05|EpIter:0.00
|  42 

In [24]:
optimizer.max

{'target': 2.4638532758561262,
 'params': {'beta_sp': 30.351417232503938,
  'epsilonIterate_logit': -112.66257648129948,
  'epsilon_sp': 13.206277525958182,
  'muIncr_logit': -3.134940502447735,
  'mu_logit': 0.812883876513182,
  'sigma_sp': 400.7046516565571}}

In [25]:
inv_logit(optimizer.max["target"])

0.92156863

In [26]:
params=optimizer.max["params"]

In [27]:
softplus(params["beta_sp"])

30.351417232504005

In [28]:
softplus(params["sigma_sp"])

400.7046516565571

In [29]:
inv_logit(params["mu_logit"])

0.6927237002031744

In [30]:
inv_logit(params["muIncr_logit"])

0.04168878236567342

In [31]:
inv_logit(params["epsilonIterate_logit"])

1.1783039829753149e-49

In [32]:
softplus(params["epsilon_sp"])

13.20627936497688

In [33]:
entropy_F1(mu=0.6927,
           muIncr=0.0417,
           beta=30,
           sigma=400,
           epsilon=13.206,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.6927
Starting Iteration epsilon= 13.206
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 15658
NewMatrix Iterateblocks Total Time = 21.513736179098487


>>>>>>>>>>>>
Starting Iteration mu= 0.7343999999999999
Starting Iteration epsilon= 13.206
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 4284
NewMatrix Iterateblocks Total Time = 4.924954152666032


>>>>>>>>>>>>
Starting Iteration mu= 0.7760999999999999
Starting Iteration epsilon= 13.206
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2703
NewMatrix Iterateblocks Total Time = 3.236268292181194


>>>>>>>>>>>>
Starting Iteration mu= 0.8177999999999999
Starting Iteration epsilon= 13.206
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2074
NewMatrix Iterateblocks Total Time = 2.34780861902982


>>>>>>>>>>>>
Starting Iteration mu= 0.8594999999999998
Starting Iteration epsilon= 13.

0.92156863